In [1]:
import psutil
import humanize
import os


import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold 
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch

##Abusive Comments, Sub-task A, results##

In [7]:
abusive_test_a = pd.read_csv('ta-misogyny-test.csv',names = ['labels','text'], header = 0)
y_pred_roberta = pd.read_csv('GJG_Tamil_xlm-roBERTa.csv',names = ['text','labels'], header = 0)
y_pred_deberta = pd.read_csv('GJG_Tamil_deBERTa.csv',names = ['text','labels'], header = 0)

In [8]:
abusive_test_a

,labels,text
0,Xenophobia,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...
1,Misandry,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ
2,Counter-speech,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...
3,None-of-the-above,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...
4,None-of-the-above,பல நல்ல கருத்துக்களை சொன்ன திருவள்ளுவர் கணவன்...
...,...,...
693,None-of-the-above,சினிவாசன் அய்யாக்கு நன்றி...
694,None-of-the-above,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை
695,None-of-the-above,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...
696,Misogyny,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...


In [9]:
y_pred_roberta

,text,labels
0,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,None-of-the-above
1,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
2,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,Counter-speech
3,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,Hope-Speech
4,பல நல்ல கருத்துக்களை சொன்ன திருவள்ளுவர் கணவன்...,Counter-speech
...,...,...
693,சினிவாசன் அய்யாக்கு நன்றி...,None-of-the-above
694,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை,None-of-the-above
695,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...,Counter-speech
696,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...,Misandry


In [10]:
y_pred_deberta

,text,labels
0,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,None-of-the-above
1,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
2,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,None-of-the-above
3,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,None-of-the-above
4,பல நல்ல கருத்துக்களை சொன்ன திருவள்ளுவர் கணவன்...,Misandry
...,...,...
693,சினிவாசன் அய்யாக்கு நன்றி...,None-of-the-above
694,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை,None-of-the-above
695,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...,None-of-the-above
696,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...,None-of-the-above


In [11]:
print(sklearn.metrics.classification_report(abusive_test_a['labels'].to_numpy(),y_pred_roberta['labels'].to_numpy()))

                   precision    recall  f1-score   support

   Counter-speech       0.28      0.49      0.36        47
       Homophobia       0.00      0.00      0.00         8
      Hope-Speech       0.43      0.12      0.18        26
         Misandry       0.65      0.63      0.64       127
         Misogyny       0.41      0.33      0.37        48
None-of-the-above       0.78      0.82      0.80       416
      Transphobic       0.00      0.00      0.00         2
       Xenophobia       0.00      0.00      0.00        24

         accuracy                           0.66       698
        macro avg       0.32      0.30      0.29       698
     weighted avg       0.65      0.66      0.65       698



In [13]:
print(sklearn.metrics.classification_report(abusive_test_a['labels'].to_numpy(),y_pred_deberta['labels'].to_numpy()))

                   precision    recall  f1-score   support

   Counter-speech       0.27      0.09      0.13        47
       Homophobia       0.27      0.38      0.32         8
      Hope-Speech       0.00      0.00      0.00        26
         Misandry       0.47      0.63      0.54       127
         Misogyny       0.50      0.06      0.11        48
None-of-the-above       0.70      0.83      0.76       416
      Transphobic       0.00      0.00      0.00         2
       Xenophobia       0.00      0.00      0.00        24

         accuracy                           0.62       698
        macro avg       0.28      0.25      0.23       698
     weighted avg       0.56      0.62      0.57       698



##Abusive Comments, Sub-task B, results##

In [17]:
abusive_test_b = pd.read_csv('ta-en-misogyny-test.csv',names = ['labels','text'], header = 0)
y_pred_roberta = pd.read_csv('GJG_TamilEnglish_xlm-roBERTa.csv',names = ['text','labels'], header = 0)
y_pred_deberta = pd.read_csv('GJG_TamilEnglish_deBERTa.csv',names = ['text','labels'], header = 0)

In [18]:
abusive_test_b

,labels,text
0,Misogyny,Yella idangalilum pengalai asinga paduthuvathe...
1,Misandry,Pannikku porandha paradhesi H Raja... Ponna pu...
2,None-of-the-above,Ayya white T-shirt! Tamil culture pathi mulusa...
3,None-of-the-above,Madu vandhi bjb fraudulent lady
4,Misandry,புரட்சியாளன் சுபா deiii sarakku midukku nnu p...
...,...,...
1851,None-of-the-above,Avanga sonanga pari 8:50 sema
1852,None-of-the-above,Bro Avanga great bro semma character antha akk...
1853,None-of-the-above,NAGENDRAN NAGA hii Ur watsapp number
1854,Counter-speech,Nengal solvathu sari endral angalum kuda vethi...


In [19]:
y_pred_roberta

,text,labels
0,Yella idangalilum pengalai asinga paduthuvathe...,None-of-the-above
1,Pannikku porandha paradhesi H Raja... Ponna pu...,Misandry
2,Ayya white T-shirt! Tamil culture pathi mulusa...,None-of-the-above
3,Madu vandhi bjb fraudulent lady,None-of-the-above
4,புரட்சியாளன் சுபா deiii sarakku midukku nnu p...,Misandry
...,...,...
1851,Avanga sonanga pari 8:50 sema,None-of-the-above
1852,Bro Avanga great bro semma character antha akk...,Hope-Speech
1853,NAGENDRAN NAGA hii Ur watsapp number,None-of-the-above
1854,Nengal solvathu sari endral angalum kuda vethi...,Counter-speech


In [20]:
y_pred_deberta

,text,labels
0,Yella idangalilum pengalai asinga paduthuvathe...,None-of-the-above
1,Pannikku porandha paradhesi H Raja... Ponna pu...,Misandry
2,Ayya white T-shirt! Tamil culture pathi mulusa...,None-of-the-above
3,Madu vandhi bjb fraudulent lady,None-of-the-above
4,புரட்சியாளன் சுபா deiii sarakku midukku nnu p...,Misogyny
...,...,...
1851,Avanga sonanga pari 8:50 sema,None-of-the-above
1852,Bro Avanga great bro semma character antha akk...,Hope-Speech
1853,NAGENDRAN NAGA hii Ur watsapp number,None-of-the-above
1854,Nengal solvathu sari endral angalum kuda vethi...,None-of-the-above


In [21]:
print(sklearn.metrics.classification_report(abusive_test_b['labels'].to_numpy(),y_pred_roberta['labels'].to_numpy()))

                   precision    recall  f1-score   support

   Counter-speech       0.32      0.49      0.38        88
       Homophobia       0.42      0.29      0.34        56
      Hope-Speech       0.39      0.26      0.31        70
         Misandry       0.62      0.72      0.67       292
         Misogyny       0.12      0.05      0.07        57
None-of-the-above       0.84      0.87      0.85      1140
      Transphobic       0.29      0.03      0.06        58
       Xenophobia       0.60      0.55      0.57        95

         accuracy                           0.72      1856
        macro avg       0.45      0.41      0.41      1856
     weighted avg       0.70      0.72      0.70      1856



In [22]:
print(sklearn.metrics.classification_report(abusive_test_b['labels'].to_numpy(),y_pred_deberta['labels'].to_numpy()))

                   precision    recall  f1-score   support

   Counter-speech       0.38      0.52      0.44        88
       Homophobia       0.48      0.43      0.45        56
      Hope-Speech       0.35      0.29      0.31        70
         Misandry       0.66      0.70      0.68       292
         Misogyny       0.20      0.19      0.20        57
None-of-the-above       0.84      0.85      0.84      1140
      Transphobic       0.44      0.24      0.31        58
       Xenophobia       0.70      0.60      0.65        95

         accuracy                           0.72      1856
        macro avg       0.51      0.48      0.49      1856
     weighted avg       0.72      0.72      0.72      1856

